# Imports necessary packages and predefined values

In [9]:
import galois # Need to install galois package: pip install galois
import numpy as np
from convert_alist import readAlist 

# ## [Option 1] The following are the parameters of the self-dual code included in the GO03_self_dual/ directory
# alistDirPath = "./alistMats/GO03_self_dual/"
# length_dist_dict = {4:2, 6:2, 8:2, 10:2, 12:4, 14:4, 16:4, 18:4, 20:4, 22:6, 24:8, 26:6, 28:6, 30:6, 32:8, 34:6, 36:8, 38:8, 40:8, 42:8, 44:8, 46:8, 48:8, 50:8, 52:10, 54:8, 56:10, 58:10, 60:12, 62:10, 64:10}
# alistEnd = ".alist"
# if_self_dual = True

### [Option 2] The following are the parameters of the *dual-containing* [n, (n+1)//2, d] QR code directory QR_dual_containing/
alistDirPath = "./alistMats/QR_dual_containing/" 
length_dist_dict = {7:3, 17:5, 23:7, 47:11, 79:15, 103:19, 167:23} 
alistEnd = ".alist"
if_self_dual = False

# Import parity check matrices of classical self-dual/dual-containing codes from alist files
The self-dual codes are contructed based on [[GO03]](https://www.sciencedirect.com/science/article/pii/S107157970300011X).

The dual-containing codes are Quadratic Residue codes, whose length is a prime number.
<!-- [1] P. Gaborit and A. Otmani, “Experimental constructions of self-dual codes,” Finite Fields and Their Applications, vol. 9, no. 3, pp. 372–394, July 2003. -->
 

## Vary the $n$ for a different length and Create a CSS code

In [ ]:
for n in length_dist_dict.keys(): # length should be chosen from the keys of length_dist_dict above
    d = length_dist_dict[n]
    F2 = galois.GF(2)
    if if_self_dual:
        print("Constructing a self-dual CSS code from [n=%s,k=%s,d=%s] self-dual code" % (n, n//2, d))
        alistFilePath = alistDirPath + "n" + str(n) + "_d" + str(d) + alistEnd
        GenMat = F2(readAlist(alistFilePath))
        G_punctured = GenMat[:, :-1]  # Puncture the last column
        Hx = G_punctured.null_space() # Parity-check matrix of the punctured code = generator matrix of the dual of the punctured code
        Hz = Hx
    else:
        if n == 17:
            alistFilePath = alistDirPath + "n" + str(n) + "_d" + str(d) + "_color.alist"
            Hx = F2(readAlist(alistFilePath)).null_space()
            Hz = Hx
        else:
            alistFilePath = alistDirPath + "n" + str(n) + "_d" + str(d) + alistEnd
            GenMat = F2(readAlist(alistFilePath))
            print("Constructing a self-dual CSS code from [n=%s,k=%s,d=%s] dual-containing code" % (GenMat.shape[1], GenMat.shape[0], d))
            H_punctured = GenMat.null_space()  # For GenMat being a dual-containing code, directly use the null-space of the generator matrix directly as the parity-check matrix
            Hx = H_punctured
            Hz = H_punctured

    comm = Hx @ Hz.T
    if np.all(comm == 0):
        print("X and Z checks commute.")
    else:
        print("X and Z checks do not commute.")
    H_CSS = np.block([
        [Hx, np.zeros_like(Hx)],
        [np.zeros_like(Hx), Hz]
    ]) 

NameError: name 'GenMat' is not defined

## Doubling Construction for Triorthogonal Codes
Take a generator matrix $G_2$ of $C_2$ which defines the $X$-stablizers of a self-dual CSS code ($k=1$) with distance $d+2$, and a generator matrix $G_{2;tri}$ of $C_{2;tri}$ which defines the $X$-stablizers of a triorthogonal CSS code ($k=1$) with distance $d$. The $X$-stablizers (and also $Z$-stablizers) of a new triorthogonal CSS code ($k=1$) with distance $d+2$ are defined by $
\begin{bmatrix}
G_2 & G_2 & \\
&& G_{2;tri}\\
& 1 & 1
\end{bmatrix}\ .$
The $X$- (and $Z$-) logicals of the new triorthogonal CSS code are defined by the all $1$ vector, i.e., the new triorthogonal admits a strongly transversal logical $X$- (and $Z$-) gate.

In [2]:
# def doubling_construction(C1)
import galois
import numpy as np

In [8]:
n_tri = 15
d_tri = 3
F2 = galois.GF(2)
alistDirPath_tri = "./alistMats/JA25_triorthogonal/" 
alistEnd = ".alist"
alistFilePath_tri = alistDirPath_tri + "n" + str(n_tri) + "_d" + str(d_tri) + "_Hx" + alistEnd
Hx = F2(readAlist(alistFilePath_tri))

alistFilePath_tri = alistDirPath_tri + "n" + str(n_tri) + "_d" + str(d_tri) + "_Hz" + alistEnd
Hz = F2(readAlist(alistFilePath_tri))
# print("Number of X checks:", Hx.shape[0], "\nNumber of Z checks:", Hz.shape[0])

comm = Hx @ Hz.T
print("Triorthogonal component: n=%s, k=%s, d=%s" % (n_tri, n_tri-Hx.shape[0]-Hz.shape[0], d_tri))
if np.all(comm == 0):
    print("  X and Z checks commute.")
else:
    print("  X and Z checks do NOT commute.")

G2_tri = Hx 

### The desired the distance of the component self-dual code should be d_tri + 2
d_sd = d_tri + 2

### Get the G2 of the self-dual CSS code
alistDirPath_sd = "./alistMats/QR_dual_containing/" 
dist_length_dict = {3:7, 5:17, 7:23, 9:45, 11:47, 15:79, 19: 103, 23:167} 
alistEnd = ".alist"
n_sd = dist_length_dict[d_sd]

alistFilePath_sd = alistDirPath_sd + "n" + str(n_sd) + "_d" + str(d_sd) + alistEnd

G2_sd = F2(readAlist(alistFilePath_sd)).null_space()
print("Self-dual component: n=%s, k=%s, d=%s" % (n_sd, n_sd-2*G2_sd.shape[0], d_sd))


### Construct the Hx for triorthogonal codes with d_tri+2 = d_sd
Hx_tri = F2(np.block([
    [G2_sd, G2_sd, np.zeros((G2_sd.shape[0], G2_tri.shape[1]), dtype=int)],
    [np.zeros((G2_tri.shape[0], G2_sd.shape[1]), dtype=int), np.zeros((G2_tri.shape[0], G2_sd.shape[1]), dtype=int), G2_tri],
    [np.zeros((1, G2_sd.shape[1]), dtype=int), np.ones((1, G2_sd.shape[1]), dtype=int), np.ones((1, G2_tri.shape[1]), dtype=int)]
    # [np.ones((1, G2_sd.shape[1]), dtype=int), np.zeros((1, G2_sd.shape[1]), dtype=int), np.ones((1, G2_tri.shape[1]), dtype=int)]
]))

Hz_dual = F2(np.vstack((np.ones((1, Hx_tri.shape[1]), dtype=int), Hx_tri)))
Hz_tri = Hz_dual.null_space()
# print("Hz shape:", Hz_tri.shape)
# print("Hx shape:", Hx_tri.shape)

comm = Hx_tri @ Hz_tri.T
n_tri_new = Hx_tri.shape[1]
d_tri_new = d_tri + 2
print("New Triorthogonal code: n=%s, k=%s, d=%s" % (n_tri_new, n_tri_new-Hx_tri.shape[0]-Hz_tri.shape[0], d_tri_new))
# # print(comm)
if np.all(comm == 0):
    print("  X and Z checks commute.")
else:
    print("  X and Z checks do NOT commute.")

Triorthogonal component: n=15, k=1, d=3
  X and Z checks commute.
Self-dual component: n=17, k=1, d=5
New Triorthogonal code: n=49, k=1, d=5
  X and Z checks commute.


In [ ]:
### The desired the distance of the component self-dual code should be d_tri + 2
d_sd = d_tri + 2

### Get the G2 of the self-dual CSS code
alistDirPath_sd = "./alistMats/QR_dual_containing/" 
dist_length_dict = {3:7, 7:23, 9: 11:47, 15:79, 19: 103, 23:167} 
alistEnd = ".alist"
n_sd = dist_length_dict[d_sd]

alistFilePath_sd = alistDirPath_sd + "n" + str(n_sd) + "_d" + str(d_sd) + alistEnd

G2_sd = F2(readAlist(alistFilePath_sd)).null_space()
print("Self-dual component: n=%s, k=%s, d=%s" % (n_sd, n_sd-2*G2_sd.shape[0], d_sd))


### Construct the Hx for triorthogonal codes with d_tri+2 = d_sd
Hx_tri = np.block([
    [G2_sd, G2_sd, np.zeros((G2_sd.shape[0], G2_tri.shape[1]))],
    [np.zeros((G2_tri.shape[0], G2_sd.shape[1])), np.zeros((G2_tri.shape[0], G2_sd.shape[1])), G2_tri],
    [np.zeros((1, G2_sd.shape[1])), np.ones((1, G2_sd.shape[1])), np.ones((1, G2_tri.shape[1]))]
])
Hz_tri = Hx_tri

comm = Hx_tri @ Hz_tri.T
n_tri_new = Hx_tri.shape[1]
print("New Triorthogonal code: n=%s, k=%s, d=%s" % (n_tri_new, n_tri_new-2*Hx_tri.shape[0], d_sd))
# print(comm)
if np.all(comm == 0):
    print("  X and Z checks commute.")
else:
    print("  X and Z checks do NOT commute.")

Self-dual component: n=23, k=1, d=7
New Triorthogonal code: n=63, k=23, d=7
  X and Z checks do NOT commute.
